## Постановка задачи
Загрузим данные, приведем их к числовым, заполним пропуски, нормализуем данные и оптимизируем память.

Разделим выборку на обучающую/проверочную в соотношении 80/20.

Построим последовательный ансамбль решающих деревьев, используя облегченный градиентный бустинг (LightGBM). Используем перекрестную проверку, чтобы найти наилучшие параметры ансамбля.

Проведем предсказание и проверим качество через каппа-метрику.

Данные:
* https://video.ittensive.com/machine-learning/prudential/train.csv.gz

Соревнование: https://www.kaggle.com/c/prudential-life-insurance-assessment/

© ITtensive, 2020

In [1]:
GRAIN = 11
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix, make_scorer
from sklearn.model_selection import GridSearchCV, cross_val_score
import lightgbm as lgb
from sklearn import preprocessing
from etl_utils import reduce_mem_usage


data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")

data['Product_Info_2_1'] = data['Product_Info_2'].str.slice(0, 1)
data['Product_Info_2_2'] = pd.to_numeric(data['Product_Info_2'].str.slice(1, 2))
data = data.drop('Product_Info_2', axis='columns')

onehot_df = pd.get_dummies(data['Product_Info_2_1'])
onehot_df.columns = ['Product_Info_2_1' + column for column in onehot_df.columns]
data = pd.merge(left=data, right=onehot_df, left_index=True, right_index=True).drop('Product_Info_2_1', axis=1).fillna(-1)
del onehot_df

data['Response'] = data['Response'] - 1

### Набор столбцов для расчета

In [2]:
columns_groups = ['Insurance_History', 'InsurеdInfo', 'Medical_Keyword', 'Family_Hist', 'Medical_History', 'Product_Info']
columns = ['Wt', 'Ht', 'Ins_Age', 'BMI']
for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print(columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keyword_31', 'Medical_Keyword_32', 'Medical_Keyword_33', 'Medical_Keyword_34', 'Medical_Keyword_35', 'Medical_Keyword_36', 'M

### Нормализация данных и Разделение данных

In [3]:
scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(data[columns]))
columns_transformed = data_transformed.columns
data_transformed['Response'] = data['Response']
data_transformed = reduce_mem_usage(data_transformed)

data_train, data_test = train_test_split(data_transformed, test_size=.2, random_state=GRAIN)
print(data_train.shape, data_test.shape)

Потребление памяти меньше на 40.49 Мб (-75.1%)
(47504, 119) (11877, 119)


### LightGBM
Основное отличие этого градиентного бустинга от предыдущих - использование сильно-разнородных (определяется разностью, гистограммой самих данных) экземпляров в выборке для формирования первоначального дерева: сначала рассматриваются все крайние, "плохие", случаи, а затем к ним "достраиваются" средние, "хорошие". Это позволяет еще быстрее минимизировать ошибку моделей.

Из дополнительных плюсов: алгоритм запускается сразу на всех ядрах процессора, это существенно ускоряет работу.

In [4]:
model = lgb.LGBMRegressor(random_state=GRAIN, max_depth=18, min_child_samples=19, num_leaves=34)

Также возможно провести классификации множества классов через LightGBM. В этом случае модель вернет вероятности принадлежности к каждому классу, возвращенные значения нужно будет дополнительно обработать через argmax, чтобы получить единственное значение класса.

In [5]:
'''model = lgb.LGBMRegressor(random_state=17, max_depth=17,
                min_child_samples=18, num_leaves=34,
                objective="multiclass", num_class=8)'''

'model = lgb.LGBMRegressor(random_state=17, max_depth=17,\n                min_child_samples=18, num_leaves=34,\n                objective="multiclass", num_class=8)'

Диапазон тестирования параметров модели ограничен только вычислительной мощностью. Для проверки модели имеет смысл провести индивидуальные перекрестные проверки для каждого параметра в отдельности, затем в итоговой проверке перепроверить самые лучшие найденные параметры с отклонением +/-10%.

Проверку качества по каппа метрике при оптимизации выполнить не удастся из-за нецелых значений Light GBM. Гиперпараметры оптимизации:
* max_depth - максимальная глубина деревьев,
* num_leaves - число листьев в каждом
* min_child_samples - минимальное число элементов выборке в листе

In [6]:
lgb_params = {
    'max_depth': range(16, 19),
    'num_leaves': range(34, 37),
    'min_child_samples': range(17, 20)
}
grid = GridSearchCV(model, lgb_params, cv=5, n_jobs=-1, verbose=True)

In [7]:
%%time
grid.fit(data_train[columns_transformed], data_train['Response'])

Fitting 5 folds for each of 27 candidates, totalling 135 fits
CPU times: total: 6.7 s
Wall time: 20.9 s


GridSearchCV(cv=5,
             estimator=LGBMRegressor(max_depth=18, min_child_samples=19,
                                     num_leaves=34, random_state=11),
             n_jobs=-1,
             param_grid={'max_depth': range(16, 19),
                         'min_child_samples': range(17, 20),
                         'num_leaves': range(34, 37)},
             verbose=True)

Выведем самые оптимальные параметры и построим итоговую модель, используя 1000 последовательных деревьев.

In [8]:
print(grid.best_params_)
model = lgb.LGBMRegressor(
    random_state=17,
    max_depth=grid.best_params_['max_depth'],
    min_child_samples=grid.best_params_['min_child_samples'],
    num_leaves=grid.best_params_['num_leaves'],
    n_estimators=1000,
    objective='multiclass', num_class=8
)

{'max_depth': 16, 'min_child_samples': 18, 'num_leaves': 36}


In [9]:
%%time
model.fit(data_train[columns_transformed], data_train['Response'])

CPU times: total: 4min 16s
Wall time: 17.4 s


LGBMRegressor(max_depth=16, min_child_samples=18, n_estimators=1000,
              num_class=8, num_leaves=36, objective='multiclass',
              random_state=17)

### Предсказание данных и оценка модели
LightGBM возвращает дробное значение класса, его нужно округлить.

Для multiclass используем argmax

In [10]:
def calculate_model(x):
    return np.argmax(model.predict([x]))

In [11]:
data_test['target'] = data_test[columns_transformed].apply(calculate_model, axis='columns')
print("LightGBM:", round(cohen_kappa_score(data_test["target"], data_test["Response"], weights="quadratic"), 3))

LightGBM: 0.551


Кластеризация дает 0.192, kNN(100) - 0.3, лог. регрессия - 0.512/0.496, SVM - 0.95, реш. дерево - 0.3, случайный лес - 0.487, XGBoost - 0.534, градиентный бустинг - 0.545

### Матрица неточностей

In [12]:
print("LightGBM\n", confusion_matrix(data_test["target"], data_test["Response"]))

LightGBM
 [[ 288  164   34   32   59  128   54   36]
 [ 215  345   25    9  106  122   40   30]
 [  14   24   56   18    0    0    0    0]
 [  39   27   52  198    0    3    0    1]
 [  91  131    7    1  570  110   10   10]
 [ 244  262   20   19  189 1162  260  164]
 [ 130  121    4    7   51  270  695  197]
 [ 217  233    4   32   89  441  503 3514]]
